In [30]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import tensorflow_datasets as tfds
tf.enable_eager_execution() 

ImportError: This version of TensorFlow Datasets requires TensorFlow version >= 2.1.0; Detected an installation of version 1.14.0. Please upgrade TensorFlow to proceed.

In [2]:
origdf = pd.read_csv('origdataset.csv')
origdf

,Unnamed: 0,account,text,party
0,0,SenShelby,Thank you to each and every one of the brave m...,R
1,1,SenShelby,I’d like to wish the @USMC a happy 245th birth...,R
2,2,SenShelby,Our Republic relies on the integrity of our el...,R
3,3,SenShelby,"Time after time, we have demonstrated our will...",R
4,4,SenShelby,"By &amp; large, these bills are the product of...",R
...,...,...,...,...
8847,8847,RepGwenMoore,Joe Biden will follow the science and data to ...,D
8848,8848,RepGwenMoore,I am so excited for the future of our country....,D
8849,8849,RepGwenMoore,"For years, Republicans have used disingenuous ...",D
8850,8850,RepGwenMoore,The voice of the American people is a powerful...,D


In [3]:
df = origdf.dropna()

In [4]:
# process text
import string

removable_punctuation = "!\"$%&'()*+,-./:;<=>?[\]^_`{|}~"
translator = str.maketrans('', '', removable_punctuation)

def remove_links(text):
    index = text.find('https://t.co/')
    if index is -1:
        return text
    else:
        text = text[0:index] + text[index+23:]
        return remove_links(text)
 

def preprocessor(text):
    try:
        text = remove_links(text)
#         print(text[index+23:])
    except:
        print(text)
    text = text.replace("&amp;", '').lower()
    return text.translate(translator)
    
# preprocessor(df.iloc[0]['text'])
df['text'] = df['text'].apply(preprocessor)

df

C:\Users\Quincy\anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,account,text,party
0,0,SenShelby,thank you to each and every one of the brave m...,R
1,1,SenShelby,i’d like to wish the @usmc a happy 245th birth...,R
2,2,SenShelby,our republic relies on the integrity of our el...,R
3,3,SenShelby,time after time we have demonstrated our willi...,R
4,4,SenShelby,by large these bills are the product of bipar...,R
...,...,...,...,...
8847,8847,RepGwenMoore,joe biden will follow the science and data to ...,D
8848,8848,RepGwenMoore,i am so excited for the future of our country ...,D
8849,8849,RepGwenMoore,for years republicans have used disingenuous f...,D
8850,8850,RepGwenMoore,the voice of the american people is a powerful...,D


In [21]:
target = df['party']
ds = tf.data.Dataset.from_tensor_slices(( df['text'],target))
for ex in ds.take(3):
    tf.print(ex[0].numpy()[:50], ex[1])

# for element in ds:
#     print(element[1])

b'thank you to each and every one of the brave men a' "R"
b'i\xe2\x80\x99d like to wish the @usmc a happy 245th birthda' "R"
b'our republic relies on the integrity of our electo' "R"


In [25]:
tf.random.set_random_seed(1)
ds = ds.shuffle(8729, reshuffle_each_iteration=False)
for ex in ds.take(3):
    tf.print(ex[0].numpy()[:50], ex[1])
    
ds_raw_test = ds.take(4000)
ds_raw_train_valid = ds.skip(4000)
ds_raw_train = ds_raw_train_valid.take(3000)
ds_raw_valid = ds_raw_train_valid.skip(3000)

b'excited to join @petebuttigieg @jenniferwebbfl and' "D"
b'new yorkers eligible to receive a stimulus payment' "D"
b'art has the power to stir awe and wonder in our he' "D"


In [28]:
from collections import Counter

tokenizer = tfds.features.text.Tokenizer()

NameError: name 'tfds' is not defined